## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,15.82,79,20,8.05,few clouds
1,1,Vista Hermosa,MX,19.5167,-99.2167,76.55,33,40,8.05,scattered clouds
2,2,Kodiak,US,57.7900,-152.4072,42.31,49,1,3.44,clear sky
3,3,Busselton,AU,-33.6500,115.3333,46.04,80,47,8.79,scattered clouds
4,4,Vestmannaeyjar,IS,63.4427,-20.2734,43.68,69,55,12.24,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp)
                                      & (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Vista Hermosa,MX,19.5167,-99.2167,76.55,33,40,8.05,scattered clouds
6,6,Puerto Ayora,EC,-0.7393,-90.3518,75.16,91,90,8.01,overcast clouds
11,11,Atuona,PF,-9.8000,-139.0333,78.04,67,16,15.66,few clouds
17,17,Avarua,CK,-21.2078,-159.7750,75.25,78,40,9.22,light rain
27,27,Kapoeta,SS,4.7717,33.5903,84.06,39,35,6.67,scattered clouds
30,30,Salalah,OM,17.0151,54.0924,77.09,65,0,2.30,clear sky
33,33,Vaini,TO,-21.2000,-175.2000,77.16,83,75,9.22,broken clouds
35,35,Abonnema,NG,4.7084,6.7931,78.42,83,86,5.53,light rain
38,38,Xuddur,SO,4.1213,43.8894,76.98,68,69,6.91,broken clouds
43,43,Canavieiras,BR,-15.6750,-38.9472,80.56,67,32,9.17,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                215
City                   215
Country                215
Lat                    215
Lng                    215
Max Temp               215
Humidity               215
Cloudiness             215
Wind Speed             215
Current Description    215
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# preferred_cities_df.replace("", nan_value, inplace=True)
# preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Vista Hermosa,MX,76.55,scattered clouds,19.5167,-99.2167,
6,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
11,Atuona,PF,78.04,few clouds,-9.8000,-139.0333,
17,Avarua,CK,75.25,light rain,-21.2078,-159.7750,
27,Kapoeta,SS,84.06,scattered clouds,4.7717,33.5903,
30,Salalah,OM,77.09,clear sky,17.0151,54.0924,
33,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
35,Abonnema,NG,78.42,light rain,4.7084,6.7931,
38,Xuddur,SO,76.98,broken clouds,4.1213,43.8894,
43,Canavieiras,BR,80.56,scattered clouds,-15.6750,-38.9472,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!=""]

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))